модель пс
модель бс
тест что инвариантны
тест что воспроизводят либраскаловскую штуковину

In [1]:
'''!wget "https://archive.materialscloud.org/record/file?file_id=b612d8e3-58af-4374-96ba-b3551ac5d2f4&filename=methane.extxyz.gz&record_id=528" -O methane.extxyz.gz
!gunzip -k methane.extxyz.gz'''

'!wget "https://archive.materialscloud.org/record/file?file_id=b612d8e3-58af-4374-96ba-b3551ac5d2f4&filename=methane.extxyz.gz&record_id=528" -O methane.extxyz.gz\n!gunzip -k methane.extxyz.gz'

In [1]:
import numpy as np
from rascal.representations import SphericalExpansion as SPH
import ase.io
import tqdm
from nice.blocks import *
from nice.utilities import *
from matplotlib import pyplot as plt
from sklearn.linear_model import BayesianRidge
import torch
torch.set_num_threads(1)
import torch.nn
from code_pytorch import *
import time
import numpy as np
from sympy.physics.wigner import wigner_3j
from sympy.physics.quantum.spin import Rotation as rot
from rascal.representations import SphericalInvariants as SOAP
from torch2trt import torch2trt

In [2]:
LAMBDA_MAX = 5

In [3]:
HYPERS = {
    'interaction_cutoff': 6.3,
    'max_radial': 5,
    'max_angular': LAMBDA_MAX,
    'gaussian_sigma_type': 'Constant',
    'gaussian_sigma_constant': 0.05,
    'cutoff_smooth_width': 0.3,
    'radial_basis': 'GTO'
}

In [4]:
structures = ase.io.read('methane.extxyz', index='0:100')

In [5]:
all_species = get_all_species(structures)

coefficients = get_spherical_expansion(structures, HYPERS,
                                             all_species)

print(coefficients.keys())
print(coefficients[1].shape)

100%|██████████| 2/2 [00:00<00:00, 1746.54it/s]

dict_keys([1, 6])
(400, 10, 6, 11)


In [6]:
def convert_to_torch(coefficients):
    result = {}
    for lambd in range(coefficients.shape[2]):
        result[lambd] = torch.FloatTensor(coefficients[:, :, lambd, : 2 * lambd + 1])
    return result

In [7]:
coefficients[1] = convert_to_torch(coefficients[1])
coefficients[6] = convert_to_torch(coefficients[6])

In [8]:
clebsch = nice.clebsch_gordan.ClebschGordan(LAMBDA_MAX)

In [9]:
class Powerspectrum(torch.nn.Module):
    def __init__(self, clebsch):
        super(Powerspectrum, self).__init__()
        self.first = ClebschCombining(clebsch, 0)       
            
    def forward(self, X):
        ps_invariants = self.first(X, X)       
        return ps_invariants

In [10]:
model_ps = Powerspectrum(clebsch.precomputed_)

In [11]:
result = model_ps(coefficients[6])
print(result.keys())
print(result[0].shape)

dict_keys([0])
torch.Size([100, 256, 1])


In [12]:
class Bispectrum(torch.nn.Module):
    def __init__(self, clebsch, lambda_max):
        super(Bispectrum, self).__init__()
        self.first = ClebschCombining(clebsch, lambda_max)
        self.second = ClebschCombining(clebsch, 0)
            
    def forward(self, X):
        ps_covariants = self.first(X, X)
        bs_invariants = self.second(ps_covariants, X)
        return bs_invariants

In [13]:
model_bs = Bispectrum(clebsch.precomputed_, LAMBDA_MAX)

In [14]:
result = model_bs(coefficients[6])

In [15]:
print(result.keys())
print(result[0].shape)

dict_keys([0])
torch.Size([100, 17408, 1])


In [16]:
class Trispectrum(torch.nn.Module):
    def __init__(self, clebsch, lambda_max):
        super(Trispectrum, self).__init__()
        self.first = ClebschCombining(clebsch, lambda_max)
        self.second = ClebschCombining(clebsch, lambda_max)
        self.third = ClebschCombining(clebsch, lambda_max)
            
    def forward(self, X):
        ps_covariants = self.first(X, X)
        bs_covariants = self.second(ps_covariants, X)
        ts_invariants = self.third(bs_covariants, X)
        return ts_invariants

In [17]:
model_ts = Trispectrum(clebsch.precomputed_, LAMBDA_MAX)

In [18]:
'''small = {key : coefficients[6][key][0:3] for key in coefficients[6].keys()}
result = model_ts(small)'''

'small = {key : coefficients[6][key][0:3] for key in coefficients[6].keys()}\nresult = model_ts(small)'

In [19]:
'''print(result.keys())
print(result[0].shape)'''

'print(result.keys())\nprint(result[0].shape)'

In [20]:
model = ClebschCombining(clebsch.precomputed_, 2)
result = model(coefficients[6], coefficients[6])
for key in result.keys():
    print(key, result[key].shape)

0 torch.Size([100, 256, 1])
1 torch.Size([100, 576, 3])
2 torch.Size([100, 704, 5])


In [21]:
HYPERS_PS = copy.deepcopy(HYPERS)
HYPERS_PS['soap_type'] = 'PowerSpectrum'

In [22]:
soap_ps = SOAP(**HYPERS_PS)
begin = time.time()
ps = soap_ps.transform(nice.rascal_coefficients.process_structures(structures)).get_features(soap_ps)
end = time.time()
print(end - begin)

0.03153800964355469


In [23]:
print(ps.shape)

(500, 192)


In [24]:
begin = time.time()
result = model_ps(coefficients[6])
result = model_ps(coefficients[1])
end = time.time()
print(end - begin)

0.017012357711791992


In [25]:
HYPERS_BS = copy.deepcopy(HYPERS)
HYPERS_BS['soap_type'] = 'BiSpectrum'
HYPERS_BS['inversion_symmetry'] = False

In [26]:
soap_bs = SOAP(**HYPERS_BS)
begin = time.time()
bs = soap_bs.transform(nice.rascal_coefficients.process_structures(structures)).get_features(soap_bs)
end = time.time()
print(end - begin)
print(bs.shape)

1.3435180187225342
(500, 8704)


In [28]:
begin = time.time()
with torch.no_grad():
    result = model_bs(coefficients[1])
    result = model_bs(coefficients[6])
end = time.time()
print(end - begin)
print(result[0].shape)

1.7501306533813477
torch.Size([100, 17408, 1])


In [29]:
model_traced = torch.jit.script(model_bs, coefficients[1])

/home/pozdn/.local/lib/python3.6/site-packages/torch/jit/_script.py:891: UserWarning: `optimize` is deprecated and has no effect. Use `with torch.jit.optimized_execution() instead
  "`optimize` is deprecated and has no effect. Use `with torch.jit.optimized_execution() instead"


RuntimeError: 
Arguments for call are not valid.
The following variants are available:
  
  aten::_set_item.t(t[](a!) l, int idx, t(b -> *) el) -> (t[](a!)):
  Could not match type Dict[str, Tensor] to List[t] in argument 'l': Cannot match List[t] to Dict[str, Tensor].
  
  aten::_set_item.str(Dict(str, t)(a!) l, str(b -> *) idx, t(c -> *) v) -> ():
  Expected a value of type 'str' for argument 'idx' but instead found type 'int'.
  
  aten::_set_item.int(Dict(int, t)(a!) l, int(b -> *) idx, t(c -> *) v) -> ():
  Expected a value of type 'Dict[int, t]' for argument 'l' but instead found type 'Dict[str, Tensor]'.
  
  aten::_set_item.bool(Dict(bool, t)(a!) l, bool(b -> *) idx, t(c -> *) v) -> ():
  Expected a value of type 'Dict[bool, t]' for argument 'l' but instead found type 'Dict[str, Tensor]'.
  
  aten::_set_item.float(Dict(float, t)(a!) l, float(b -> *) idx, t(c -> *) v) -> ():
  Expected a value of type 'Dict[float, t]' for argument 'l' but instead found type 'Dict[str, Tensor]'.

In [29]:
'''begin = time.time()
result = model_ts(coefficients[6])
end = time.time()
print(end - begin)
print(result[0].shape)'''

'begin = time.time()\nresult = model_ts(coefficients[6])\nend = time.time()\nprint(end - begin)\nprint(result[0].shape)'

In [33]:
for key in coefficients[1].keys():
    coefficients[1][key] = coefficients[1][key].cuda()
for key in coefficients[6].keys():
    coefficients[6][key] = coefficients[6][key].cuda()

In [34]:
model_ps = model_ps.cuda()

In [35]:
begin = time.time()
result = model_ps(coefficients[6])
end = time.time()
print(end - begin)

0.0015459060668945312


In [36]:
model_bs = model_bs.cuda()

In [42]:
begin = time.time()
result = model_bs(coefficients[1])
result = model_bs(coefficients[6])
end = time.time()
print(end - begin)
print(result[0].is_cuda)

0.0739743709564209
True


In [45]:
model_bs_tensorrt = torch2trt(model_bs, [coefficients[1]])

AttributeError: 'dict' object has no attribute 'clone'

In [35]:
print(result[0].is_cuda)

True


In [36]:
print(type(coefficients[1][1].device))

<class 'torch.device'>


In [37]:
print(str(coefficients[1][1].device) == 'cpu')

False


In [38]:
str(coefficients[1][1].device)

'cuda:0'

In [39]:
a = torch.FloatTensor([1, 2, 3])
print(str(a.device))

cpu


In [40]:
print(a.is_cuda)

False


In [41]:
print(coefficients[1][1].is_cuda)

True


In [46]:
from rascal.models import Kernel, SparsePoints, train_gap_model

In [48]:
HYPERS_PS_GRADIENTS = copy.deepcopy(HYPERS_PS)
HYPERS_PS_GRADIENTS['compute_gradients'] = True

In [50]:
soap_ps = SOAP(**HYPERS_PS_GRADIENTS)
begin = time.time()
ps = soap_ps.transform(nice.rascal_coefficients.process_structures(structures)).get_features(soap_ps)
end = time.time()
print(end - begin)
print(ps.shape)

0.05592036247253418
(500, 192)


In [52]:
soap_ps = SOAP(**HYPERS_PS)
begin = time.time()
ps = soap_ps.transform(nice.rascal_coefficients.process_structures(structures)).get_features(soap_ps)
end = time.time()
print(end - begin)
print(ps.shape)

0.03480672836303711
(500, 192)


In [ ]:
SOAP()


In [53]:
ob = soap_ps.transform(nice.rascal_coefficients.process_structures(structures))

In [54]:
ob.managers

In [57]:
HYPERS_GRAD = copy.deepcopy(HYPERS)
HYPERS_GRAD['compute_gradients'] = True

In [58]:
 sph = SPH(**HYPERS)

In [61]:
feat = sph.transform(nice.rascal_coefficients.process_structures(structures)).get_features(sph)

In [62]:
print(feat.shape)

(500, 128)


In [63]:
ob = sph.transform(nice.rascal_coefficients.process_structures(structures))

In [ ]:
ob.get_features()